## proglie

In [11]:
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneOut
from sklearn import metrics    # contains all the effectivness idexes 
import pandas as pd    # for the dataset
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# setting up labels for dataset
labels = ['class', 'spec_num', 'eccentr', 'asp_ratio', 'elong', 'solidity', 'stoch_conv', 'iso_factor', 'max_ind_depth', 'lobedness', 'av_intensity', 'av_contr', 'smooth', 'third_mom', 'unif', 'entropy']
# importing data
df = pd.read_csv(r'./leaf/leaf.csv', header = None, names = labels)
# shuffling the dataframe
df = df.sample(frac=1).reset_index()
df = df.iloc[:, 1:17]   # needed to eliminate the old indexes column
# separating y from x and eliminating specimen number variable
X = df.iloc[:, 2:16]
y = df.iloc[:, 0]

### random forest

we first use grid sarch with k-fold cross validation (for n_var we choose sqrt(p) and p/3, with p total number variable)

In [3]:
k = 5

# griglia dei parametri su cui fare la ricerca
grid_param = {"n_estimators": (100, 200, 500, 700), 'criterion': ('gini', 'entropy'), 'max_features': ('sqrt', 5)}

rf_cv = GridSearchCV(RandomForestClassifier(), grid_param, cv=k, scoring='balanced_accuracy')
rf_cv.fit(X, y)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ('gini', 'entropy'),
                         'max_features': ('sqrt', 5),
                         'n_estimators': (100, 200, 500, 700)},
             refit=False, scoring='balanced_accuracy')

In [ ]:
results_rf_cv = pd.DataFrame(rf_cv.cv_results_)

display(results_rf_cv)

we now try with leave-one-out cross validation

In [ ]:
grid_param = {"n_estimators": (100, 200, 500, 700), 'criterion': ('gini', 'entropy'), 'max_features': ('sqrt', 5)}

rf_loocv = GridSearchCV(RandomForestClassifier(), grid_param, cv=LeaveOneOut(), scoring='accuracy', n_jobs=5)
rf_loocv.fit(X, y)

In [ ]:
results_rf_loocv = pd.DataFrame(rf_loocv.cv_results_)

display(results_rf_loocv)

### single tree

grid search with k-fold cross validation

In [12]:
k = 5

# griglia dei parametri su cui fare la ricerca
grid_param = {'criterion': ['gini', 'entropy'], 'min_samples_split': np.arange(2, 20)}

tree_cv = GridSearchCV(tree.DecisionTreeClassifier(), grid_param, cv=k, scoring='balanced_accuracy')
tree_cv.fit(X, y)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'min_samples_split': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19])},
             scoring='balanced_accuracy')

In [ ]:
results_tree_cv = pd.DataFrame(tree_cv.cv_results_)

display(results_tree_cv)

grid search with leave-one-out cross validation

In [14]:
tree_loocv = GridSearchCV(tree.DecisionTreeClassifier(), grid_param, cv=LeaveOneOut(), scoring='accuracy')
tree_loocv.fit(X, y)

GridSearchCV(cv=LeaveOneOut(), estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'min_samples_split': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19])},
             scoring='accuracy')

In [ ]:
results_tree_loocv = pd.DataFrame(tree_loocv.cv_results_)

display(results_tree_loocv)

### SVM

grid search with k-fold cross validation